In [1]:
# Evaluate model predictions on "val" set

#from ultralytics import YOLO
import os
import pandas as pd
import yaml
import matplotlib.pyplot as plt

## Specify paths here:

#GROUND
# text_files_dir = "/home/cara/oregon_critters/runs/detect/7_mc_ground_med_rep3/predict/labels"
# yaml_file_path = "/home/cara/oregon_critters/dataset_ground.yaml"
# output_csv_path = "/home/cara/oregon_critters/runs/detect/7_mc_ground_med_rep3/predict/pred_boxes_07_mc_ground_med_rep3.csv"

#TRAIL
# text_files_dir = "/home/cara/oregon_critters/trail/runs/detect/8_mc_trail_med_rep3/predict/labels"
# yaml_file_path = "/home/cara/oregon_critters/trail/dataset_trail.yaml"
# output_csv_path = "/home/cara/oregon_critters/trail/runs/detect/8_mc_trail_med_rep3/predict/pred_boxes_08_mc_trail_med_rep3.csv"

#BOTH on GROUND
# text_files_dir = "/home/cara/oregon_critters/both/runs/detect/9_mc_both_med_rep3/predict_on_ground/labels"
# yaml_file_path = "/home/cara/oregon_critters/both/dataset_ground_both.yaml"
# output_csv_path = "/home/cara/oregon_critters/both/runs/detect/9_mc_both_med_rep3/predict_on_ground/pred_boxes_09_mc_med_on_ground_rep3.csv"

#BOTH on TRAIL
# text_files_dir = "/home/cara/oregon_critters/both/runs/detect/9_mc_both_med_rep3/predict_on_trail/labels"
# yaml_file_path = "/home/cara/oregon_critters/both/dataset_trail_both.yaml"
# output_csv_path = "/home/cara/oregon_critters/both/runs/detect/9_mc_both_med_rep3/predict_on_trail/pred_boxes_09_mc_med_on_trail_rep3.csv"

#BOTH on BOTH
text_files_dir = "/home/cara/oregon_critters/both/runs/detect/9_mc_both_med_rep3/predict_on_both/labels"
yaml_file_path = "/home/cara/oregon_critters/both/dataset_both.yaml"
output_csv_path = "/home/cara/oregon_critters/both/runs/detect/9_mc_both_med_rep3/predict_on_both/pred_boxes_09_mc_med_on_both_rep3.csv"


In [2]:
# Initialize an empty list to store the data
data = []

# Iterate through each text file in the directory
for file_name in os.listdir(text_files_dir):
    file_path = os.path.join(text_files_dir, file_name)
    with open(file_path, "r") as file:
        lines = file.readlines()
        
        # Process each line (row) in the file
        for line in lines:
            values = line.strip().split()
            data.append([file_name] + values)

# Create dataframe 
columns = ["filename"] + [f"value_{i+1}" for i in range(len(data[0])-1)]
preds = pd.DataFrame(data, columns=columns)

# Create column names
column_names = ["image_name"] + ["class_ID"] + ["x"] + ["y"]+ ["w"]+ ["h"]+ ["confidence"]
preds.columns = column_names

# Remove .txt from image name
preds['image_name'] = preds.image_name.map(lambda x: x.strip('.txt'))
preds.head()

,image_name,class_ID,x,y,w,h,confidence
0,10171-4__2019-08-12__14-30-39(1)_hy,35,0.529152,0.59626,0.0450098,0.129255,0.751028
1,10171-4__2019-08-12__14-30-39(1)_hy,40,0.528197,0.595545,0.0462043,0.129464,0.540266
2,10171-4__2019-08-12__14-30-39(1)_hy,11,0.234623,0.806769,0.0992505,0.117277,0.113518
3,10171-4__2019-08-12__14-30-40(2)_hy,35,0.622081,0.572257,0.0496962,0.0988031,0.582348
4,10171-4__2019-08-12__14-30-40(2)_hy,11,0.23339,0.806662,0.0981074,0.116252,0.536835


In [3]:
## Match class IDs with actual class names

# Read in original yaml file
with open(yaml_file_path, 'r') as file:
    yaml_data = yaml.safe_load(file)

# Extract just the "names" data from the yaml and create list of dictionaries
names_data = yaml_data.get('names', {})
data = [{'class_ID': key, 'class_pred': value} for key, value in names_data.items()]

# Create dataframe
class_ids = pd.DataFrame(data)

# Convert column 'class_ID' to string
class_ids['class_ID'] = class_ids['class_ID'].astype(str)

# Merge with "preds" dataframe
preds=preds.merge(class_ids, how='left', on='class_ID')
preds.head()

# Save
preds.to_csv(output_csv_path)